In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import cv2
from os.path import join
import os
import random
from functools import partial
from keras.models import Model, load_model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Reshape, concatenate
from keras.optimizers import Nadam
from keras.layers.advanced_activations import ELU
from keras.layers.normalization import BatchNormalization
from keras import backend as K
from keras.backend import binary_crossentropy

import tensorflow as tf

K.clear_session()
K.set_image_dim_ordering('tf')

print(K.image_data_format())
smooth = 1e-12
num_examples = 0

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


channels_last


In [2]:
import glob
import ntpath
#image_size = 512
def load_files():
    global image_size
    global num_examples
    output_mask_path = "masks/training/mask1/"
    original_path = "masks/training/ori/"
    mask_path = "unet_train/gt/*"
    masks = glob.glob(mask_path)
    print("Loading masks")
    for fl in masks:
        train_image = cv2.imread(fl)
        shape = train_image.shape
        #Supervisely gives 3 images in out and we need the mask only
        start = shape[1]//3
        end = 2 * start
        basenm = ntpath.basename(fl)
        nm_output_mask = output_mask_path+basenm
        train_image = train_image[: , start:end]
        #train_image = cv2.resize(train_image, (image_size, image_size))
        cv2.imwrite(nm_output_mask,train_image)

    
    orig_path = "unet_train/orig/*"
    orig = glob.glob(orig_path)
    print("loading original")
    for fl in orig:
        orig_images = cv2.imread(fl)
        #orig_images = cv2.resize(orig_images, (image_size, image_size))
        basenm = ntpath.basename(fl)
        nm_output_ori = original_path+basenm
        cv2.imwrite(nm_output_ori,orig_images)

load_files()


Loading masks
loading original


In [4]:
!ls

01.png	  4229.png				logs
1511.png  4237.png				mask.png
1610.png  4248.png				masks
1902.jpg  archive				modeltr_b2.h5
1902.png  ConvertAllJpgToPNG.ipynb		modeltr_b3.h5
1903.jpg  ConvertSupervisely3imagesinto1.ipynb	modeltr.h5
1910.jpg  Createnewmasks-batch.ipynb		output.png
4189.png  Createnewmasks-Copy1.ipynb		plot.png
4197.png  Createnewmasks-Copy2.ipynb		train
4201.png  Createnewmasks-Copy3.ipynb		Unet-batches.ipynb
4204.png  Createnewmasks-Copy4.ipynb		Unet.ipynb
4209.png  Createnewmasks.ipynb			unet_train
4217.png  hello.jpg				unet-train.ipynb
4221.png  input.png


In [11]:
r=cv2.imread('masks/training/mask1/1901.jpg')
print(sorted(set( tuple(v) for m2d in r for v in m2d )))
#backtorgb = cv2.cvtColor(r,cv2.COLOR_GRAY2RGB)
#gray_image = cv2.cvtColor(r, cv2.COLOR_BGR2GRAY)


[(0, 0, 0), (0, 0, 1), (0, 0, 3), (0, 0, 4), (0, 0, 5), (0, 0, 6), (0, 0, 7), (0, 0, 8), (0, 1, 0), (0, 1, 1), (0, 1, 2), (0, 1, 4), (0, 1, 5), (0, 1, 6), (0, 2, 0), (0, 2, 1), (0, 2, 2), (0, 2, 3), (0, 2, 5), (0, 2, 6), (0, 2, 7), (0, 3, 0), (0, 3, 1), (0, 3, 2), (0, 3, 3), (0, 3, 4), (0, 3, 6), (0, 3, 7), (0, 4, 0), (0, 4, 1), (0, 4, 2), (0, 4, 3), (0, 4, 4), (0, 4, 5), (0, 4, 7), (0, 5, 0), (0, 5, 1), (0, 5, 2), (0, 5, 3), (0, 5, 4), (0, 5, 5), (0, 5, 6), (0, 6, 0), (0, 6, 1), (0, 6, 2), (0, 6, 3), (0, 6, 4), (0, 6, 5), (0, 6, 9), (0, 7, 0), (0, 7, 1), (0, 7, 2), (0, 7, 3), (0, 7, 4), (0, 7, 5), (0, 7, 6), (0, 8, 0), (0, 8, 1), (0, 8, 2), (0, 8, 4), (0, 8, 5), (0, 8, 7), (0, 9, 0), (0, 9, 1), (0, 9, 2), (0, 9, 3), (0, 9, 6), (0, 10, 0), (0, 10, 1), (0, 10, 2), (0, 10, 3), (0, 10, 4), (0, 11, 0), (0, 11, 1), (0, 11, 2), (0, 11, 3), (0, 11, 4), (0, 12, 0), (0, 12, 1), (0, 12, 2), (0, 12, 3), (0, 12, 4), (0, 12, 5), (0, 12, 6), (0, 13, 0), (0, 13, 1), (0, 13, 2), (0, 13, 4), (0, 14, 0)

In [10]:
gray_image[0]

array([149,  23,   6, ...,   0,   3,   4], dtype=uint8)

In [12]:
r=cv2.imread('masks/training/batch_2/gen_uniform/1901.png')
print(sorted(set( tuple(v) for m2d in r for v in m2d )))

[(0, 0, 0), (126, 211, 33)]


In [13]:
r=cv2.imread('masks/training/batch_2/gen_uniform/4222.png')
print(sorted(set( tuple(v) for m2d in r for v in m2d )))

[(0, 0, 0), (126, 211, 33)]
